# Penny Stock Model

In [1]:
# import libraries
import pandas as pd
import yfinance as yf
import numpy as np
import datetime

In [2]:
# parameters
VolRise = 0.2  # open position, after current volume will be more than VolRise value
ExitAfter = 0.25 # close position after price achieve ExitAfter
StopLoss = -0.07 # close position after price meets stop
RiskPertrade = 2000 # buy for sum 
TradeAfterDays = 150 # begin trade after TradeAfterDays days from start
price_filter =0.5 # below this price we open pos


In [14]:
yf.Ticker.get_splits('REED')

AttributeError: 'str' object has no attribute '_history'

In [12]:
help(yf)

Help on package yfinance:

NAME
    yfinance

DESCRIPTION
    # -*- coding: utf-8 -*-
    #
    # yfinance - market data downloader
    # https://github.com/ranaroussi/yfinance
    #
    # Copyright 2017-2019 Ran Aroussi
    #
    # Licensed under the Apache License, Version 2.0 (the "License");
    # you may not use this file except in compliance with the License.
    # You may obtain a copy of the License at
    #
    #     http://www.apache.org/licenses/LICENSE-2.0
    #
    # Unless required by applicable law or agreed to in writing, software
    # distributed under the License is distributed on an "AS IS" BASIS,
    # WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    # See the License for the specific language governing permissions and
    # limitations under the License.
    #

PACKAGE CONTENTS
    base
    multi
    shared
    ticker
    tickers
    utils
    version

CLASSES
    builtins.object
        yfinance.tickers.Tickers
    yfinance.base.Ticker

In [10]:
list_stocks

,Deficiency,Market,NotDate
Company,,,
REED,Bid Price,NCM,2021-08-16
FWBI,Equity,NCM,2021-11-26
IDRA,Bid Price,NCM,2021-11-26
SPCB,Bid Price,NCM,2021-12-13
SECO,Bid Price,NCM,2021-12-17
...,...,...,...
XELA,Bid Price,NCM,2022-12-10
FAMI,Bid Price,NCM,2022-12-10
HUIZ,Bid Price,NGM,2022-12-10


In [3]:
# load ticker list from excel-file
list_stocks = pd.read_excel('inputFiles/list_stocks.xlsx', index_col ='Company')
all_res = []
data_group=[]
data_group_res=[]

stock_res =[]
stock_small_res=[]
#len(list_stocks)
for i in range(len(list_stocks)): # потім прописати весь список
    symbol = list_stocks.index[i] # беремо акцію по індексу
    datefromlist = list_stocks.iloc[i]['NotDate']
    print('symbol=',symbol)
    print('datefromlist=',datefromlist)
    
    # to calc average volume use 20 days before
    start_date = datefromlist - datetime.timedelta(days=19)                                           
    end_date = datefromlist + datetime.timedelta(days=180)

    #load stock price
    #symbol = list_stocks.index[i] # беремо акцію по індексу
    wdf = yf.download(symbol, start_date, end_date)



    # building moving average
    wdf['avgvol']= wdf['Volume'].rolling(window=10, win_type=None).mean()  # середній обєм за 10 днів
    wdf['VolRise'] =wdf['Volume']/wdf['avgvol']-1      # building vol signal
    wdf['volsignal']=wdf['VolRise']>VolRise            # signal column
    wdf['TradeAfter']=wdf.index

    wdf['TradeAfterDaysSig'] = wdf['TradeAfter'] > datefromlist + datetime.timedelta(days=TradeAfterDays)
    wdf['OpenPrice']=0
    wdf['Profit']=0
    wdf['StopPrice']=0
    wdf['TargetPrice']=0
    wdf['Ticker']=symbol
    del(wdf['TradeAfter'])
    lastrow=wdf.tail(1).index # last row in order to stop calculations

    # main module
    PriceOpen = 0
    StopPrice = 0
    TPPrice = 0
    CloseSignal = 0
    status = 0
    
    
    for index, row in wdf.iterrows():
        # looking for opening signal 
        if (PriceOpen ==0) & (lastrow[0]!=index):
            if row['volsignal'] & row['TradeAfterDaysSig']:  # if Volume > than AvgVolume

                PriceOpen = row['Close']  # record close price as price of open
                wdf.at[index,'OpenPrice']= PriceOpen
                StopPrice = PriceOpen-(PriceOpen*StopLoss*-1) # setting stop-loss

                TPPrice = PriceOpen+(PriceOpen*ExitAfter)  # set target price
                status=1

    #     # working with first opened position
        if (PriceOpen >0) & (status==0):
            wdf.at[index,'OpenPrice']= PriceOpen
            wdf.at[index,'StopPrice']= StopPrice
            wdf.at[index,'TargetPrice']= TPPrice


            if row['High']>TPPrice:
                wdf.at[index,'Profit']= RiskPertrade*ExitAfter
                PriceOpen =0
                TPPrice=0
                StopPrice=0

            if row['Low']<StopPrice:
                wdf.at[index,'Profit']= RiskPertrade*StopLoss
                PriceOpen =0
                TPPrice=0
                StopPrice=0


        status = 0    

        # if there is open position and its last row
        if lastrow[0]==index:
            if PriceOpen >0:
                wdf.at[index,'Profit']=(row['Close']/PriceOpen-1)*RiskPertrade
                PriceOpen =0
                TPPrice=0
                StopPrice=0
   
    
    #тут закінчується блок перебору строк акції і йде на наступну акцію
    # 1. тікер
    # 2. profit
    # 3. tradecount
    # 4. ProfitTrades
    #  5.   LossTrades
    # 6.    WinRate
    #7.     AvgHold'
    
    tradecount=(wdf['Profit']!=0).sum()
    ProfitTrades = (wdf['Profit']>0).sum()
    LossTrades = (wdf['Profit']<0).sum()
    #WinRate = round(ProfitTrades/tradecount,1)
    #AvgHold = round((wdf['OpenPrice']!=0).sum()/tradecount,1)
    stock_small_res=[symbol,round(wdf['Profit'].sum(),2),tradecount,ProfitTrades,LossTrades]
    
    data_group=wdf['Profit']
    stock_res.append(stock_small_res)
    data_group_res.append(data_group)
    # блок запису dataFrame:
    all_res.append(wdf)
      
    df_output = pd.concat(all_res)
    
    df_data =pd.concat(data_group_res)
    #df_data.columns['Profit']
   # mask1 = df_data.Profit >0
    #mask2 = df_data.Profit <0
    #df_out= df_data.loc[df_data['Profit']!=0]



    
      
        

symbol= REED
datefromlist= 2021-08-16 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= FWBI
datefromlist= 2021-11-26 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= IDRA
datefromlist= 2021-11-26 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= SPCB
datefromlist= 2021-12-13 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= SECO
datefromlist= 2021-12-17 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= ECOR
datefromlist= 2021-12-20 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= SNGX
datefromlist= 2021-12-20 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= ASTC
datefromlist= 2021-12-21 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= CHEK
datefromlist= 2021-12-23 00:00:00
[*********************100

[*********************100%***********************]  1 of 1 completed
symbol= SINT
datefromlist= 2022-03-01 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= SCPS
datefromlist= 2022-03-03 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= EAST
datefromlist= 2022-03-06 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= EZGO
datefromlist= 2022-03-06 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= HCTI
datefromlist= 2022-03-06 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= HEPA
datefromlist= 2022-03-06 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= RETO
datefromlist= 2022-03-06 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= WINT
datefromlist= 2022-03-06 00:00:00
[*********************100%***********************]  1 of 1 completed
sym

[*********************100%***********************]  1 of 1 completed
symbol= PSTV
datefromlist= 2022-05-24 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= IPDN
datefromlist= 2022-05-24 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= SGLY
datefromlist= 2022-05-24 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= SRAX
datefromlist= 2022-05-24 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= NISN
datefromlist= 2022-05-25 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= CTIB
datefromlist= 2022-05-26 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= GFAI
datefromlist= 2022-05-27 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= ACER
datefromlist= 2022-05-31 00:00:00
[*********************100%***********************]  1 of 1 completed
sym

[*********************100%***********************]  1 of 1 completed
symbol= APTO
datefromlist= 2022-07-18 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= OIG
datefromlist= 2022-07-19 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= RVSN
datefromlist= 2022-07-19 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= RSLS
datefromlist= 2022-07-19 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= HLBZ
datefromlist= 2022-07-20 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= HLBZ
datefromlist= 2022-07-20 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= ELYS
datefromlist= 2022-07-25 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= IKT
datefromlist= 2022-07-25 00:00:00
[*********************100%***********************]  1 of 1 completed
symbo

[*********************100%***********************]  1 of 1 completed
symbol= METX
datefromlist= 2022-09-16 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= TAOP
datefromlist= 2022-09-16 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= HSDT
datefromlist= 2022-09-19 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= LITM
datefromlist= 2022-09-19 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= TOUR
datefromlist= 2022-09-19 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= NUZE
datefromlist= 2022-09-20 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= LYL
datefromlist= 2022-09-21 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= BACK
datefromlist= 2022-09-21 00:00:00
[*********************100%***********************]  1 of 1 completed
symb

[*********************100%***********************]  1 of 1 completed
symbol= TNXP
datefromlist= 2022-10-25 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= AHG
datefromlist= 2022-10-26 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= DCTH
datefromlist= 2022-10-26 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= FRGT
datefromlist= 2022-10-26 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= NLTX
datefromlist= 2022-10-26 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= PTE
datefromlist= 2022-10-26 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= SSKN
datefromlist= 2022-10-26 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= BVXV
datefromlist= 2022-10-27 00:00:00
[*********************100%***********************]  1 of 1 completed
symbo

[*********************100%***********************]  1 of 1 completed
symbol= TSP
datefromlist= 2022-11-17 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= BBIG
datefromlist= 2022-11-17 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= ADVM
datefromlist= 2022-11-18 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= AYLA
datefromlist= 2022-11-18 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= BYSI
datefromlist= 2022-11-18 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= FLGT
datefromlist= 2022-11-18 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= HTGM
datefromlist= 2022-11-18 00:00:00
[*********************100%***********************]  1 of 1 completed
symbol= NYMX
datefromlist= 2022-11-18 00:00:00
[*********************100%***********************]  1 of 1 completed
symb

In [4]:
df_output['Profit'].sum()

9814.303990880933

In [5]:
stock_df = pd.DataFrame(stock_res, columns = ['Stock','Profit','TradeCount','ProfitTrades','LossTrades'])
df_day = pd.DataFrame(df_data )
df_day.index.name ="Date"

In [6]:
df_day = df_day.loc[df_day['Profit']!=0] # remove 0 values
df_day = df_day.groupby([df_day.index.date]).sum() # групуємо по даті


In [7]:
df_day.sort_index()


,Profit
2022-01-24,-140.000000
2022-02-11,66.666600
2022-05-02,-140.000000
2022-05-10,-140.000000
2022-05-17,500.000000
...,...
2023-01-09,-140.000000
2023-01-10,580.000000
2023-01-11,360.000000
2023-01-12,1080.000000


In [8]:
with pd.ExcelWriter("outputFiles/output.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    df_output.to_excel(writer, sheet_name="full_B")
    df_day.to_excel(writer, sheet_name="by date")
    stock_df.to_excel(writer, sheet_name="statByStock")

In [9]:
df_day


,Profit
2022-01-24,-140.000000
2022-02-11,66.666600
2022-05-02,-140.000000
2022-05-10,-140.000000
2022-05-17,500.000000
...,...
2023-01-09,-140.000000
2023-01-10,580.000000
2023-01-11,360.000000
2023-01-12,1080.000000


In [22]:
testDF = yf.download("AEI", "2022-10-01", "2022-12-10",back_adjust=True)

[*********************100%***********************]  1 of 1 completed


In [23]:
testDF

,Open,High,Low,Close,Volume
Date,,,,,
2022-09-30,3.88,5.60,3.88,5.12,182160
2022-10-03,5.02,5.46,4.40,4.64,41625
2022-10-04,5.40,5.58,4.62,4.76,138730
2022-10-05,4.60,4.70,4.22,4.44,35735
2022-10-06,4.60,4.60,4.20,4.54,16660
2022-10-07,4.54,4.66,4.30,4.32,24710
2022-10-10,4.46,4.46,4.10,4.44,11785
2022-10-11,4.30,4.50,4.02,4.36,22850
2022-10-12,4.50,4.50,4.20,4.50,12395
